In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings

# Suprimir todos los warnings de pandas
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [2]:
data = pd.read_csv("data/dataraw.csv")
data.head()

,genres,title,tags,specs,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,761140,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",0.00,643980,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",0.00,670290,Poolians.com
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",['Single-player'],0.99,767400,彼岸领域
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,"['Action', 'Adventure', 'Simulation', 'FPS', '...","['Single-player', 'Steam Achievements']",3.99,772540,Trickjump Games Ltd


In [3]:
print(data.shape)
data = data.drop_duplicates(subset='id')
data = data.reset_index(drop=True)
print(data.shape)

(27170, 7)
(27169, 7)


In [4]:
### Función que realiza una transformacion conveniennte y retorna la lista unica de variables:
def unique_list(variable:str):
    # transformar las observaciones 
    data[variable]=data[variable].str.replace('[',"")
    data[variable]=data[variable].str.replace("]","")
    data[variable]=data[variable].str.replace("]","")
    data[variable]=data[variable].str.replace("'","")
    # extraer las especificaciones unicas
    unique_variable_list = list(set(data[variable]))
    unique_variable_list = [i.split(", ") for i in unique_variable_list]
    unique_variable_list = list(set([i[j] for i in unique_variable_list for j in range(len(i))]))
    return unique_variable_list

In [5]:
specs_list = unique_list('specs')
genres_list = unique_list('genres')
tags_list = unique_list('tags')

/var/folders/6s/zvsfwc9975gdw5pt_krwlbpm0000gn/T/ipykernel_10204/2788107322.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[variable]=data[variable].str.replace('[',"")
/var/folders/6s/zvsfwc9975gdw5pt_krwlbpm0000gn/T/ipykernel_10204/2788107322.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[variable]=data[variable].str.replace("]","")
/var/folders/6s/zvsfwc9975gdw5pt_krwlbpm0000gn/T/ipykernel_10204/2788107322.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[variable]=data[variable].str.replace(

In [6]:
#para especificaciones
data_sp = {data.iloc[i]['id']:data['specs'][i].split(', ') for i in range(len(data.index)) }
#para genero
data_gn = {data.iloc[i]['id']:data['genres'][i].split(', ') for i in range(len(data.index)) }
#para tags
data_tg = {data.iloc[i]['id']:data['tags'][i].split(', ') for i in range(len(data.index))}

In [7]:
variables = specs_list + genres_list + tags_list
variables.append('Precio') #añadir variable de precio
len(variables)

388

In [8]:
## crear una base de datos para las variables por juego:
df_vars = pd.DataFrame(0, index=data['id'], columns=variables) # df lleno de ceros

In [9]:
## llenar el dataframe para cada juego en 1 si cuenta con esa especificacion
counter = 0
for ID in df_vars.index:
    for columna in data_sp[ID]:
        df_vars.at[ID, columna] = 1
    for columna in data_gn[ID]:
        df_vars.at[ID, columna] = 1
    for columna in data_tg[ID]:
        df_vars.at[ID,columna] = 1
    df_vars.at[ID,'Precio'] =  float(data['price'][data['id']==ID])
    counter += 1
    if counter % 2000 == 0:
        print(counter)

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000


In [10]:
df_vars.to_csv('data/df_vars.csv')

In [11]:
## aplicar la similitud del coseno:
cs = cosine_similarity(df_vars,df_vars)
print(cs)

[[1.         0.1523567  0.2920722  ... 0.8885797  0.69216672 0.83286955]
 [0.1523567  1.         0.36514837 ... 0.18282804 0.25357428 0.12188536]
 [0.2920722  0.36514837 1.         ... 0.2503476  0.34722088 0.208623  ]
 ...
 [0.8885797  0.18282804 0.2503476  ... 1.         0.64580623 0.8885797 ]
 [0.69216672 0.25357428 0.34722088 ... 0.64580623 1.         0.64580623]
 [0.83286955 0.12188536 0.208623   ... 0.8885797  0.64580623 1.        ]]


In [14]:
import time
# Crear un nuevo DataFrame con los valores de similitud
it = time.time()
df_similarity = pd.DataFrame(cs, index=df_vars.index, columns=df_vars.index)

# Imprimir el nuevo DataFrame de similitud
ft = time.time()
et = ft - it
print(f'Tiempo en Guardar: {et} segundos')
df_similarity

Tiempo en Guardar: 0.0003199577331542969 segundos


id,761140,643980,670290,767400,772540,774276,774277,774278,770380,768880,...,80,767010,761480,767590,747320,745400,773640,733530,610660,658870
id,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.152357,0.292072,0.587271,0.773086,0.839888,0.854544,0.851700,0.895686,0.850053,...,0.842604,0.879058,0.367823,0.525140,0.849382,0.707284,0.845592,0.888580,0.692167,0.832870
643980,0.152357,1.000000,0.365148,0.064630,0.067900,0.184257,0.103076,0.066270,0.095066,0.149320,...,0.097282,0.125921,0.253379,0.231170,0.071657,0.182802,0.293188,0.182828,0.253574,0.121885
670290,0.292072,0.365148,1.000000,0.265495,0.139464,0.275241,0.153974,0.098993,0.108478,0.255581,...,0.111007,0.172425,0.208172,0.395679,0.049060,0.312890,0.468375,0.250348,0.347221,0.208623
767400,0.587271,0.064630,0.265495,1.000000,0.589147,0.353687,0.359740,0.358491,0.549287,0.503043,...,0.468095,0.402854,0.391067,0.445987,0.366796,0.794832,0.470887,0.469109,0.292054,0.469109
772540,0.773086,0.067900,0.139464,0.589147,1.000000,0.748275,0.761317,0.758776,0.804451,0.643312,...,0.806853,0.723338,0.410330,0.350233,0.751848,0.648856,0.594243,0.679981,0.616743,0.679981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745400,0.707284,0.182802,0.312890,0.794832,0.648856,0.571381,0.549732,0.534241,0.692308,0.662275,...,0.553063,0.629567,0.553741,0.631508,0.553931,1.000000,0.667270,0.707284,0.553473,0.707284
773640,0.845592,0.293188,0.468375,0.470887,0.594243,0.635497,0.601530,0.580061,0.647333,0.708128,...,0.591355,0.673155,0.443505,0.675231,0.592283,0.667270,1.000000,0.800923,0.740483,0.711584
733530,0.888580,0.182828,0.250348,0.469109,0.679981,0.855200,0.863110,0.857207,0.881202,0.850053,...,0.783318,0.879058,0.414148,0.630801,0.871217,0.707284,0.800923,1.000000,0.645806,0.888580


In [15]:

# Crear un diccionario para almacenar los resultados
similarity_dict = {}
counter = 0

st = time.time()
# Iterar sobre las filas del DataFrame de similitud
for index, row in df_similarity.iterrows():
    
    # Iniciar el temporizador
    if counter % 2000 == 0:
        start_time = time.time()
    
    # Obtener los 5 mayores valores de similitud (excluyendo el propio índice)
    top_similarities = row.drop(index).nlargest(5)

    # Crear una lista de tuplas (id, similitud)
    id_similarity_list = list(zip(top_similarities.index, top_similarities.values))

    # Almacenar la lista en el diccionario
    similarity_dict[index] = id_similarity_list
    # Imprimir el tiempo transcurrido y un conteo de ayuda
    counter += 1
    if counter % 2000 == 0:
        end_time = time.time()
        elapsed_time = end_time - st
        print('-'*15)
        print(f"Tiempo R{counter}: {elapsed_time} segundos")
        
print('='*20)
ft = time.time()
et = ft-st
print(f'Tiempo Total: {et} segundos')

---------------
Tiempo R2000: 8.614591836929321 segundos
---------------
Tiempo R4000: 16.27534794807434 segundos
---------------
Tiempo R6000: 23.723301887512207 segundos
---------------
Tiempo R8000: 31.16611886024475 segundos
---------------
Tiempo R10000: 38.57791590690613 segundos
---------------
Tiempo R12000: 46.003002882003784 segundos
---------------
Tiempo R14000: 53.5530469417572 segundos
---------------
Tiempo R16000: 62.080183029174805 segundos
---------------
Tiempo R18000: 70.07666778564453 segundos
---------------
Tiempo R20000: 77.44716882705688 segundos
---------------
Tiempo R22000: 84.8160047531128 segundos
---------------
Tiempo R24000: 92.22745394706726 segundos
---------------
Tiempo R26000: 99.62361478805542 segundos
Tiempo Total: 104.2148699760437 segundos


In [16]:
import json
# Ruta del archivo JSON donde se guardará los resultados del modelo de recomendacion
ruta_archivo_json = "data/item_item_recomendation_system.json"

# Guardar el diccionario como un archivo JSON
with open(ruta_archivo_json, 'w') as archivo_json:
    json.dump(similarity_dict, archivo_json)

print(f"El diccionario se ha guardado como {ruta_archivo_json}")

El diccionario se ha guardado como data/item_item_recomendation_system.json
